In [ ]:
import ccxt
import time
import pandas as pd
import numpy as np
from datetime import datetime, timezone
from ta.momentum import RSIIndicator
from ta.trend import MACD
from ta.volatility import BollingerBands, AverageTrueRange
from tensorflow.keras.models import load_model
import joblib
from collections import Counter
import warnings
import os
import signal

# --- Global Flag for Interruption ---
stop_signal = False

def keyboard_interrupt_handler(signum, frame):
    """Sets the global flag to True upon catching Ctrl+C."""
    global stop_signal
    stop_signal = True
    print("\n\n!! Ctrl+C detected. Preparing to stop prediction loop. !!")

# Set the signal handler for reliable interrupt detection
signal.signal(signal.SIGINT, keyboard_interrupt_handler)


# --- Custom Sleep Function ---
def interruptible_sleep(seconds):
    """Sleeps for the given duration but checks for interrupt every second."""
    global stop_signal
    for _ in range(int(seconds)):
        if stop_signal:
            return True # Interrupted
        time.sleep(1)
    return stop_signal

# Suppress minor warnings for cleaner output
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# ----------------------------
# Load saved models and preprocessors
# ----------------------------
try:
    forest_model = joblib.load('forest_model.pkl')
    xgb_model = joblib.load('xgb_model.pkl')
    label_encoder = joblib.load('label_encoder.pkl')
    scaler_tabular = joblib.load('scaler_tabular.pkl')
    scaler_long = joblib.load('scaler_long.pkl')
    neural_model = load_model('neural_model.keras', compile=False)
except FileNotFoundError as e:
    print(f"Error: One or more model files not found. Please ensure all required files are in the directory. Missing: {e}")
    exit()

# ----------------------------
# Define Activity Helpers (DUMMY IMPLEMENTATION)
# ----------------------------

def buy_col(row):
    if 'rsi_6' in row and row['rsi_6'] < 30:
        return 1
    return 0

def sell_col(row):
    if 'rsi_6' in row and row['rsi_6'] > 70:
        return 1
    return 0

def define_activity(row):
    if row['buy_index'] > row['sell_index']:
        return 'BUY'
    elif row['sell_index'] > row['buy_index']:
        return 'SELL'
    return 'HOLD'


# ----------------------------
# Configuration and Helper Functions
# ----------------------------
binance = ccxt.binance()
symbol = 'BTC/USDT'
timeframe = '15m'
limit_historical = 200
prediction_interval_sec = 5 * 60 # 5 minutes

def get_current_candle_start_ms(tf):
    now_ms = int(datetime.now(timezone.utc).timestamp() * 1000)

    unit = tf[-1]
    value = int(tf[:-1])

    if unit == 'm':
        interval_ms = value * 60 * 1000
    elif unit == 'h':
        interval_ms = value * 60 * 60 * 1000
    else:
        raise ValueError(f"Timeframe {tf} not supported for calculation.")

    start_ms = now_ms - (now_ms % interval_ms)
    return start_ms

def clear_console():
    if os.name == 'nt':
        os.system('cls')
    else:
        os.system('clear')

# ----------------------------
# 1. Fetch Historical Context (Closed Candles Only)
# ----------------------------
since_dt = datetime.now(timezone.utc) - pd.Timedelta(days=7)
since = int(since_dt.timestamp() * 1000)

all_candles = []
clear_console()
print(f"Fetching historical data for {symbol} on {timeframe}...")

while True:
    try:
        candles = binance.fetch_ohlcv(symbol, timeframe=timeframe, since=since, limit=limit_historical)
        if not candles:
            break
        all_candles.extend(candles)
        since = candles[-1][0] + 1
        time.sleep(0.1)
        if len(candles) < limit_historical:
             break
    except Exception as e:
        print(f"Error fetching historical data: {e}. Retrying in 5 seconds.")
        time.sleep(5)
        break
    if stop_signal:
        print("\nStopping historical data fetch.")
        exit()


historical_df = pd.DataFrame(all_candles, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
historical_df['timestamp'] = pd.to_datetime(historical_df['timestamp'], unit='ms')
historical_df = historical_df.tail(limit_historical)


# =========================================================
# LIVE PREDICTION LOOP
# =========================================================
clear_console()
print("="*60)
print(f"🚀 Starting Real-Time Prediction for {symbol} / {timeframe}")
print(f"   Re-predicting every {prediction_interval_sec / 60:.0f} minutes...")
print("   Press Ctrl+C to stop.")
print("="*60)

while not stop_signal:
    try:
        clear_console()

        # --- 2. Get Live Price and Construct Forming Candle Data ---

        last_closed_candle = historical_df.iloc[-1]
        ticker = binance.fetch_ticker(symbol)
        live_price = ticker['last']
        current_candle_start_ms = get_current_candle_start_ms(timeframe)
        current_candle_start_dt = pd.to_datetime(current_candle_start_ms, unit='ms')

        current_open = last_closed_candle['close']
        current_high = max(live_price, current_open)
        current_low = min(live_price, current_open)
        current_volume = 0

        new_row = pd.DataFrame({
            'timestamp': [current_candle_start_dt],
            'open': [current_open],
            'high': [current_high],
            'low': [current_low],
            'close': [live_price],
            'volume': [current_volume]
        })

        # --- 3. Combine Data and Recalculate Features ---

        df = pd.concat([historical_df, new_row]).drop_duplicates(subset=['timestamp'], keep='last').sort_values('timestamp').reset_index(drop=True)

        # ----------------------------
        # Feature engineering
        # ----------------------------
        df['rsi_6'] = RSIIndicator(df['close'], window=6).rsi()
        df['rsi_12'] = RSIIndicator(df['close'], window=12).rsi()

        macd_indicator = MACD(close=df['close'], window_slow=26, window_fast=12, window_sign=9)
        df['macd'] = macd_indicator.macd()
        df['macd_signal'] = macd_indicator.macd_signal()
        df['macd_hist'] = macd_indicator.macd_diff()

        df['ema_21'] = df['close'].ewm(span=21, adjust=False).mean()
        df['sma_50'] = df['close'].rolling(window=50).mean()
        df['ema/sma crossover'] = df['ema_21'] - df['sma_50']
        df['trends with sma'] = df['close'] / df['sma_50'] - 1

        df['price change'] = np.log(df['close'] / df['close'].shift(1))

        df['buy_index'] = df.apply(buy_col, axis=1)
        df['sell_index'] = df.apply(sell_col, axis=1)
        scale = 1
        df['signal_scaled'] = 0.5 * (np.tanh((df['buy_index'] - df['sell_index']) / scale) + 1)
        df['activity'] = df.apply(define_activity, axis=1)

        bollinger = BollingerBands(close=df['close'], window=20, window_dev=2)
        df['bollinger_hband'] = bollinger.bollinger_hband()
        df['bollinger_lband'] = bollinger.bollinger_lband()
        df['bollinger_mavg'] = bollinger.bollinger_mavg()
        df['bollinger_bandwidth'] = bollinger.bollinger_wband()

        atr = AverageTrueRange(high=df['high'], low=df['low'], close=df['close'], window=14)
        df['atr'] = atr.average_true_range()

        df['volume_change'] = df['volume'].pct_change()
        df['volume_sma_10'] = df['volume'].rolling(window=10).mean()

        df.dropna(inplace=True)
        df.set_index('timestamp', inplace=True)

        # --- 4. Select features and Predict on the Latest Row ---

        if df.empty:
            print("Dataframe is empty after dropping NaNs. Waiting for more data.")
            interruptible_sleep(prediction_interval_sec)
            continue

        feature_cols = [c for c in df.columns if c not in ['date', 'next_high', 'next_low', 'buy_index', 'sell_index', 'signal', 'signal_scaled', 'activity']]
        X = df[feature_cols]
        X_latest = X.iloc[[-1]]

        # Scale features
        X_tabular_scaled = scaler_tabular.transform(X_latest)
        X_long_scaled = scaler_long.transform(X_latest)

        # Get individual model predictions and probabilities
        proba_rf = forest_model.predict_proba(X_tabular_scaled) # RandomForest probabilities
        proba_xgb = xgb_model.predict_proba(X_tabular_scaled)
        proba_nn = neural_model.predict(X_long_scaled)

        # Get predicted classes
        pred_rf = label_encoder.inverse_transform(np.argmax(proba_rf, axis=1))
        pred_xgb = label_encoder.inverse_transform(np.argmax(proba_xgb, axis=1))
        pred_nn = label_encoder.inverse_transform(np.argmax(proba_nn, axis=1))

        # Majority voting
        votes = [pred_rf[0], pred_xgb[0], pred_nn[0]]
        voted_pred = Counter(votes).most_common(1)[0][0]

        class_names = label_encoder.classes_

        # ----------------------------------------------------
        # 🔥 NEW: ENSEMBLE CONFIDENCE CHECK
        # ----------------------------------------------------
        FINAL_PREDICTION = voted_pred
        CONFIDENCE_THRESHOLD = 0.2 # Using 10% from the user input

        # Get the average probability for each class across all three models
        # Ensure probas are arrays and handle potential 1D output from RF/XGB if applicable (though predict_proba is usually 2D)

        # Map class names to column indices for consistent ordering
        class_indices = {name: i for i, name in enumerate(class_names)}

        # Initialize dictionary to hold summed probabilities
        ensemble_probas = {name: 0.0 for name in class_names}

        # Sum probabilities from all three models
        for name in class_names:
            idx = class_indices[name]
            ensemble_probas[name] += proba_rf[0][idx]
            ensemble_probas[name] += proba_xgb[0][idx]
            ensemble_probas[name] += proba_nn[0][idx]
            # Average: Divide by the number of models (3)
            ensemble_probas[name] /= 3.0

        # Convert dictionary to list of (class, average_proba) tuples and sort
        sorted_probas = sorted(ensemble_probas.items(), key=lambda item: item[1], reverse=True)

        # Extract top two average probabilities
        top_signal_proba = sorted_probas[0][1]
        second_signal_proba = sorted_probas[1][1]

        # Check the margin
        confidence_margin = top_signal_proba - second_signal_proba

        if confidence_margin < CONFIDENCE_THRESHOLD:
            FINAL_PREDICTION = 'HOLD'
            reasoning = f"Ensemble Margin ({confidence_margin:.4f}) is below threshold ({CONFIDENCE_THRESHOLD:.4f}). Overridden to HOLD."
        else:
            reasoning = "Ensemble confidence margin is sufficient."

        # --- 5. Print Results ---

        print("="*50)
        print(f"PREDICTING for CANDLE: {timeframe} / {symbol}")
        print(f"  Current UTC Time:  {datetime.now(timezone.utc).strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"  Candle Start Time: {current_candle_start_dt.strftime('%H:%M:%S')} (Forming)")
        print(f"  Current Price:   {live_price:.2f} USDT")
        print("—"*50)

        print(f"Majority Voted Signal: {voted_pred}")
        print(f"RandomForest Prediction: {pred_rf[0]}")
        print(f"XGBoost Prediction:      {pred_xgb[0]}")
        print(f"NeuralNet Prediction:    {pred_nn[0]}")

        # Print ALL probabilities
        print("\n— Individual Model Probabilities —")
        for i, name in enumerate(class_names):
            print(f"  {name}: RF={proba_rf[0][i]:.4f} | XGB={proba_xgb[0][i]:.4f} | NN={proba_nn[0][i]:.4f}")

        print("\n— Ensemble Average Probabilities —")
        for name, proba in sorted_probas:
            print(f"  {name}: {proba:.4f}")

        print("\n" + "="*50)
        print(f"Confidence Margin:     {confidence_margin:.4f} (Required: {CONFIDENCE_THRESHOLD:.4f})")
        print(f"Decision Reason:       {reasoning}")
        print(f"🚨 FINAL TRADING SIGNAL: {FINAL_PREDICTION} 🚨")
        print("="*50)

        # ----------------------------
        # 6. Wait for the next prediction interval (Interruptible!)
        # ----------------------------
        if interruptible_sleep(prediction_interval_sec):
            break

    except Exception as e:
        print(f"\nAn error occurred in the prediction loop: {e}")
        if interruptible_sleep(prediction_interval_sec * 3):
             break

print("\nPrediction loop gracefully stopped.")

Fetching historical data for BTC/USDT on 15m...
🚀 Starting Real-Time Prediction for BTC/USDT / 15m
   Re-predicting every 5 minutes...
   Press Ctrl+C to stop.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
PREDICTING for CANDLE: 15m / BTC/USDT
  Current UTC Time:  2025-10-13 12:27:33
  Candle Start Time: 12:15:00 (Forming)
  Current Price:   114262.47 USDT
——————————————————————————————————————————————————
Majority Voted Signal: Sell
RandomForest Prediction: Sell
XGBoost Prediction:      Sell
NeuralNet Prediction:    Sell

— Individual Model Probabilities —
  Buy: RF=0.3160 | XGB=0.1648 | NN=0.3678
  Sell: RF=0.6840 | XGB=0.8352 | NN=0.6322

— Ensemble Average Probabilities —
  Sell: 0.7172
  Buy: 0.2828

Confidence Margin:     0.4343 (Required: 0.2000)
Decision Reason:       Ensemble confidence margin is sufficient.
🚨 FINAL TRADING SIGNAL: Sell 🚨
